In [ ]:
!pip install datasets

In [2]:
from datasets import load_dataset

# 假设你的 JSON 文件名为 'train.json'
test_dataset = load_dataset('json', data_files='test.json')

In [46]:
def clean_string(s):
    # 去掉首尾的 $
    if s.startswith('$') and s.endswith('$'):
        s = s[1:-1]
    
    # 去掉所有空格
    s = s.replace(' ', '')
    
    return s

In [190]:
import openai

# Create an instance of the OpenAI client
client = openai.OpenAI(api_key="sk-1HFzyn0XMWoAbfXRB69568A1A2844285A3C97d84CcD8DcDf", base_url="https://api.xiaoai.plus/v1")

def get_response(question):
    # stream = client.chat.completions.create(
    #     model="gpt-3.5-turbo",
    #     messages=[{"role": "user", "content": question[0]}],
    #     stream=True,
    # )

    # for chunk in stream:
    #     if chunk.choices[0].delta.content is not None:
    #         response = chunk.choices[0].delta.content
    #         print(response, end="")
    # print('\n')
    
    conversation_history = []
    
    for q in question:
        # print(q)
        conversation_history.append({"role": "user", "content": q}) 
        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=conversation_history,
            stream=True,
        )
        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                response_chunk = chunk.choices[0].delta.content
                response += response_chunk
                print(response_chunk, end="")
        print('\n')
        conversation_history.append({"role": "assistant", "content": response})

    return conversation_history[-1]["content"]

reference = '''{
        "text": "椭圆$\\frac{x^{2}}{2}+\\frac{y^{2}}{3}=1$的离心率为?",
        "fact_expressions": "G: Ellipse;Expression(G) = (x^2/2 + y^2/3 = 1)",
        "query_expressions": "Eccentricity(G)",
        "answer_expressions": "sqrt(3)/3",
        "fact_spans": "[[[0, 37]], [[0, 37]]]",
        "query_spans": "[[[0, 43]]]",
        "process": ""
    }'''

count = 0
true, false = [], []
for item in test_dataset['train']:
    if 'Eccentricity' in item['query_expressions']:
        count += 1
        # question = ["参考reference的格式，回答question。\n reference:"+reference+"\n question:"+item['text'],
        #             "请确认answer_expressions并输出，不需要任何其他内容"]
        questions = ["双曲线$x^{2}-y^{2}=1$的离心率为? 请给出解决这个问题的sympy代码"]
        answer = get_response(questions)
        break
        true_answer = item['answer_expressions']
        print(item['text'])

        if clean_string(true_answer) == clean_string(answer):
            true.append({'text':item['text']})
        else:
            false.append({'text':item['text'], 'answer':clean_string(answer), 'true_answer':clean_string(true_answer)})
# print(count)
# print(len(true))
# print(len(false))

为了计算双曲线$x^{2}-y^{2}=1$的离心率，我们可以使用sympy来求解。

```python
import sympy as sp

# 定义变量
x, y = sp.symbols('x y')

# 定义双曲线方程
eq = sp.Eq(x**2 - y**2, 1)

# 将方程改写成标准形式
# x = cosh(t), y = sinh(t)
# cosh(t)^2 - sinh(t)^2 = 1
# cosh(t)^2 - (cosh(t)^2 - 1) = 1
# 1 - cosh(t)^2 = 1
# cosh(t)^2 - 1 = -1
# cosh(t)^2 = 0
# cosh(t) = 0
# t = 0
# x = cosh(0) = 1, y = sinh(0) = 0
# 因此将双曲线转换成参数方程
# x = cosh(t), y = sinh(t)

# 定义参数
t = sp.symbols('t')

# 将双曲线方程使用参数表示
x_parametric = sp.cosh(t)
y_parametric = sp.sinh(t)

# 计算离心率
eccentricity = sp.sqrt(x_parametric**2 - y_parametric**2)
print("The eccentricity of the hyperbola is:", eccentricity)
```

运行代码后，将得到双曲线的离心率。



In [191]:
import sympy as sp

# 定义变量
x, y = sp.symbols('x y')

# 定义双曲线方程
eq = sp.Eq(x**2 - y**2, 1)

# 将方程改写成标准形式
# x = cosh(t), y = sinh(t)
# cosh(t)^2 - sinh(t)^2 = 1
# cosh(t)^2 - (cosh(t)^2 - 1) = 1
# 1 - cosh(t)^2 = 1
# cosh(t)^2 - 1 = -1
# cosh(t)^2 = 0
# cosh(t) = 0
# t = 0
# x = cosh(0) = 1, y = sinh(0) = 0
# 因此将双曲线转换成参数方程
# x = cosh(t), y = sinh(t)

# 定义参数
t = sp.symbols('t')

# 将双曲线方程使用参数表示
x_parametric = sp.cosh(t)
y_parametric = sp.sinh(t)

# 计算离心率
eccentricity = sp.sqrt(x_parametric**2 - y_parametric**2)
print("The eccentricity of the hyperbola is:", eccentricity)

The eccentricity of the hyperbola is: sqrt(-sinh(t)**2 + cosh(t)**2)


In [189]:
import openai
import time

# 创建 OpenAI 客户端实例
client = openai.OpenAI(api_key="sk-1HFzyn0XMWoAbfXRB69568A1A2844285A3C97d84CcD8DcDf", base_url="https://api.xiaoai.plus/v1")

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = []
    
    for q in question:
        conversation_history.append({"role": "user", "content": q}) 
        
        retries = 0
        while retries < max_retries:
            try:
                # 从模型中流式获取响应
                stream = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=conversation_history,
                    stream=True,
                )
                response = ""
                for chunk in stream:
                    if 'choices' in chunk and 'delta' in chunk.choices[0] and 'content' in chunk.choices[0].delta:
                        response_chunk = chunk.choices[0].delta['content']
                        response += response_chunk

                conversation_history.append({"role": "assistant", "content": response})
                return response

            except openai.error.OpenAIError as e:
                if '502' in str(e):
                    print(f"Encountered a 502 error. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    print(f"An unexpected error occurred: {e}")
                    break
        
        if retries == max_retries:
            print("Max retries reached. Unable to get a response.")
            return None

reference = '''{
        "text": "椭圆$\\frac{x^{2}}{2}+\\frac{y^{2}}{3}=1$的离心率为?",
        "fact_expressions": "G: Ellipse;Expression(G) = (x^2/2 + y^2/3 = 1)",
        "query_expressions": "Eccentricity(G)",
        "answer_expressions": "sqrt(3)/3",
        "fact_spans": "[[[0, 37]], [[0, 37]]]",
        "query_spans": "[[[0, 43]]]",
        "process": ""
    }'''

count = 0
true, false = [], []
for item in test_dataset['train']:
    if 'Eccentricity' in item['query_expressions']:
        count += 1
        question = ["双曲线$x^{2}-y^{2}=1$的离心率为? 请给出解决这个问题的sympy代码"]
        answer = get_response(question)
        
        true_answer = item['answer_expressions']
        print(item['text'])
        break
        
        if clean_string(true_answer) == clean_string(answer):
            true.append({'text':item['text']})
        else:
            false.append({'text':item['text'], 'answer':clean_string(answer), 'true_answer':clean_string(true_answer)})
print(count)
print(len(true))
print(len(false))


双曲线$x^{2}-y^{2}=1$的离心率为?
1
0
0


In [58]:
import json
with open('true.json', 'w', encoding='utf-8') as json_file:
    json.dump(true, json_file, indent=4)
    
with open('false.json', 'w', encoding='utf-8') as json_file:
    json.dump(false, json_file, indent=4)

In [60]:
for item in false[:5]:
    print(item)

{'text': '双曲线$x^{2}-y^{2}=1$的离心率为?', 'answer': '', 'true_answer': 'sqrt(2)'}
{'text': '点$P(x, y)$是椭圆$\\frac{x^{2}}{a^{2}} + \\frac{y^{2}}{b^{2}}=1  (a>b>0)$上的任意一点，$F_{1}$, $F_{2}$是椭圆的两个焦点，且$\\angle F_{1} P F_{2} \\leq 90^{\\circ}$，则该椭圆的离心率的取值范围是?', 'answer': '', 'true_answer': '(0,sqrt(2)/2]'}
{'text': '已知椭圆$C$: $\\frac{x^{2}}{a^{2}}+\\frac{y^{2}}{b^{2}}=1(a>b>0)$，直线$y=\\frac{\\sqrt{6}}{3} b$与椭圆$C$交于$A$、$B$两点，若$O A \\perp O B$，则椭圆离心率的值等于?', 'answer': '', 'true_answer': 'sqrt(2)/2'}
{'text': '已知椭圆$\\frac{x^{2}}{a^{2}}+\\frac{y^{2}}{b^{2}}=1(a>b>0)$的左焦点$F_{1}$、$O$为坐标原点，点$P$在椭圆上，点$Q$在椭圆的右准线上，若$\\overrightarrow{P Q}=2 \\overrightarrow{F_{1} O}$, $\\overrightarrow{F_{1} Q}=\\lambda(\\frac{\\overrightarrow{F_{1} P}}{|\\overrightarrow{F_{1} P}|}+\\frac{\\overrightarrow{F_{1} O}}{|\\overrightarrow{F_{1} O}|})(\\lambda>0)$则椭圆的离心率为?', 'answer': '', 'true_answer': '(sqrt(5)-1)/2'}
{'text': '已知双曲线$\\frac{y^{2}}{m}-x^{2}=1(m>0)$的一个焦点与抛物线$y=\\frac{1}{8} x^{2}$的焦点重合，则此双曲线的离心率为?', 'answer': '', 'true_

In [184]:
# 分类
# 【曲线类型，求解公式（离心率/方程参数/几何参数/表达式/点的横坐标/点的纵坐标/点的坐标），答案类型（简单数/复杂数/方程/区间/点坐标）】
def check_expressions_in_string(s, expressions):
    for expr in expressions:
        if expr in s:
            return True
    return False

count = 0
for item in test_dataset['train']:
    if 'Eccentricity' in item['query_expressions']:
        # 离心率
        item['answer_class'] = 'Eccentricity'
    if check_expressions_in_string(['Distance', 'Area', 'LineSegmentOf', 'Perimeter'], item['query_expressions']):
        # 几何量
        item['answer_class'] = 'Geometry'
    if 'Expression' in item['query_expressions']:
        # 曲线/直线表示
        item['answer_class'] = 'Expression'
    if check_expressions_in_string(['Coordinate'], item['query_expressions']):
        # 点坐标
        item['answer_class'] = 'Coordinate'
    if check_expressions_in_string(['AngleOf'], item['query_expressions']):
        # 点坐标
        item['answer_class'] = 'Angle'
    else:
        # 方程参数
        count+=1
        
    

{'text': '双曲线$x^{2}-y^{2}=1$的离心率为?', 'fact_expressions': 'G: Hyperbola;Expression(G) = (x^2 - y^2 = 1)', 'query_expressions': 'Eccentricity(G)', 'answer_expressions': 'sqrt(2)', 'fact_spans': '[[[0, 18]], [[0, 18]]]', 'query_spans': '[[[0, 24]]]', 'process': '由题可得a=b=1,c=\\sqrt{a^{2}+b^{2}}=\\sqrt{2},故离心率e=\\frac{c}{a}=\\sqrt{2}.'}
{'text': '点$P(x, y)$是椭圆$\\frac{x^{2}}{a^{2}} + \\frac{y^{2}}{b^{2}}=1  (a>b>0)$上的任意一点，$F_{1}$, $F_{2}$是椭圆的两个焦点，且$\\angle F_{1} P F_{2} \\leq 90^{\\circ}$，则该椭圆的离心率的取值范围是?', 'fact_expressions': 'P: Point;PointOnCurve(P, G) = True;Coordinate(P) = (x1, y1);x1: Number;y1: Number;G: Ellipse;Expression(G) = (y^2/b^2 + x^2/a^2 = 1);b: Number;a: Number;a > b;b > 0;F1: Point;F2: Point;Focus(G) = {F1, F2};AngleOf(F1, P, F2) <= ApplyUnit(90, degree)', 'query_expressions': 'Range(Eccentricity(G))', 'answer_expressions': '(0,sqrt(2)/2]', 'fact_spans': '[[[0, 10]], [[0, 73]], [[0, 10]], [[1, 10]], [[1, 10]], [[11, 67], [91, 93], [141, 143]], [[11, 67]], [[13, 67]], [[13, 6

In [192]:
import sympy as sp

# 定义变量
a, b = sp.symbols('a b')

# 计算离心率
a_value = 1
b_value = 1
eccentricity = sp.sqrt(1 + b_value**2 / a_value**2)
eccentricity


1.41421356237310

In [178]:
print(count)

2069


In [93]:
def extract_last_dollar_content(s):
    # 找到最后两个 $ 符号的位置
    last_index = s.rfind('$')
    second_last_index = s.rfind('$', 0, last_index)

    if last_index == -1 or second_last_index == -1:
        return None  # 如果找不到两个 $ 符号，返回 None

    # 提取两个 $ 符号之间的内容
    content = s[second_last_index + 1:last_index]
    return content.strip()  # 去掉首尾的空格

In [166]:
import openai
import time
# from openai.error import APIError

# 创建 OpenAI 客户端实例
client = openai.OpenAI(api_key="sk-bPPWTHPgSKQBwxqJ3e053762DaB64b969e9e68F180CfBa46", base_url="https://api.xiaoai.plus/v1")

def get_response(question, max_retries=5, wait_time=5):
    conversation_history = []
    
    for q in question:
        # print(q)
        conversation_history.append({"role": "user", "content": q}) 
        # 从模型中流式获取响应
        stream = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=conversation_history,
            stream=True,
        )
        response = ""

        for chunk in stream:
            try:
                if chunk.choices[0].delta.content is not None:
                    response_chunk = chunk.choices[0].delta.content
                    response += response_chunk
                    # print(response_chunk, end="")
            except IndexError as e:
                pass
        # print('\n')      
        conversation_history.append({"role": "assistant", "content": response})
        
    return response

true1, false1 = [], []
for item in test_dataset['train']:
    # if 'Eccentricity' in item['query_expressions']:
    question = [item['text'], 
                '''将你的最终答案转换成sympy格式并输出，只需告诉我sympy格式的答案，不需要任何解释，也不需要e=这类的符号，如果是一个取值范围，请写成区间的形式。并将答案用$包裹
                参考：$sqrt(2)$   $(0,sqrt(2)/2]$''',
                '''请确认你的答案符合格式要求。参考：$sqrt(2)$   $(0,sqrt(2)/2]$''']
    answer = get_response(question)
    
    true_answer = item['answer_expressions']

    if clean_string(true_answer) == clean_string(answer):
        true1.append({'text':item['text']})
    else:
        false1.append({'text':item['text'], 'answer':clean_string(answer), 'true_answer':clean_string(true_answer)})
    print('-'*30)
    print(true_answer)
    print(extract_last_dollar_content(answer))
        
print(count)
print(len(true1))
print(len(false1))


------------------------------
x^2/3-y^2/6=1
frac{x^{2}}{9-b^{2}} - \frac{y^{2}}{b^{2}} = 1
------------------------------
sqrt(10)
[0, \frac{3}{\sqrt{2}}]


KeyboardInterrupt: 

In [146]:
import sympy as sp

# 定义符号
x = sp.Symbol('x')

# 表达式
expr1 = '2**0.5'
expr2 = 'sqrt(2)'

# 简化表达式
simplified_expr1 = sp.simplify(expr1)
simplified_expr2 = sp.simplify(expr2)

print("Simplified expr1:", simplified_expr1)
print("Simplified expr2:", simplified_expr2)

# 将表达式转换为数值形式
evalf_expr1 = simplified_expr1.evalf()
evalf_expr2 = simplified_expr2.evalf()

print("Evalf expr1:", evalf_expr1)
print("Evalf expr2:", evalf_expr2)

# 检查表达式是否相等
are_equal = sp.simplify(simplified_expr1 - simplified_expr2) == 0
print("Are expr1 and expr2 equal:", are_equal)


Simplified expr1: 1.41421356237310
Simplified expr2: sqrt(2)
Evalf expr1: 1.41421356237310
Evalf expr2: 1.41421356237310
Are expr1 and expr2 equal: False


In [114]:
import os
from google.cloud import dialogflow_v2 as dialogflow

# 设置 Google Cloud API 密钥
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/your/service-account-file.json"

def detect_intent_texts(project_id, session_id, texts, language_code):
    session_client = dialogflow.SessionsClient()
    session = session_client.session_path(project_id, session_id)

    for text in texts:
        text_input = dialogflow.TextInput(text=text, language_code=language_code)
        query_input = dialogflow.QueryInput(text=text_input)
        response = session_client.detect_intent(session=session, query_input=query_input)
        print("Query text:", response.query_result.query_text)
        print("Detected intent:", response.query_result.intent.display_name)
        print("Detected intent confidence:", response.query_result.intent_detection_confidence)
        print("Fulfillment text:", response.query_result.fulfillment_text)

project_id = "your-project-id"
session_id = "your-session-id"
texts = ["Hello, how are you?", "Tell me about Gemini AI."]
language_code = "en"

detect_intent_texts(project_id, session_id, texts, language_code)


<module 'antlr4' from 'c:\\Users\\HUAWEI\\.conda\\envs\\nlp_wjl\\Lib\\site-packages\\antlr4\\__init__.py'>
